In [ ]:
import os
import labrotation.file_handling as fh
import datadoc_util
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches

In [ ]:
env_dict = dict()
if not os.path.exists("./.env"):
    print(".env does not exist")
else:
    with open("./.env", "r") as f:
        for line in f.readlines():
            l = line.rstrip().split("=")
            env_dict[l[0]] = l[1]
print(env_dict.keys())

In [ ]:
if "DATA_DOCU_FOLDER" in env_dict.keys():
    docu_folder = env_dict["DATA_DOCU_FOLDER"]
else:
    docu_folder = fh.open_dir("Choose folder containing folders for each mouse!")
print(f"Selected folder:\n\t{docu_folder}")

In [ ]:
if "documentation" in os.listdir(docu_folder):
    mouse_folder = os.path.join(docu_folder, "documentation")
else:
    mouse_folder = docu_folder
mouse_names = os.listdir(mouse_folder)
print(f"Mice detected:")
for mouse in mouse_names:
    print(f"\t{mouse}")

In [ ]:
ddoc = datadoc_util.DataDocumentation(docu_folder)
ddoc.loadDataDoc()

In [ ]:
df_colors = ddoc.getColorings()

In [ ]:
# filter by existing mice
df_colors = df_colors[df_colors["mouse_id"].isin(mouse_names)]

In [ ]:
df_colors["win_type"] = df_colors.apply(lambda row: ddoc.getMouseWinInjInfo(row["mouse_id"])["window_type"].iloc[0], axis=1)

In [ ]:
df_colors["inj_type"] = df_colors.apply(lambda row: ddoc.getMouseWinInjInfo(row["mouse_id"])["injection_type"].iloc[0], axis=1)

In [ ]:
df_colors = df_colors.sort_values(by=["inj_type", "win_type", "red"])

In [ ]:
# Create a figure and axis
fig = plt.figure(figsize=(28,4))

ax = plt.gca()
# Plot squares with labels
i = 0  # in the dataframe, the index has gaps
for i_bad, row in df_colors.iterrows():
    color = row['color']
    label = row['mouse_id']
    
    # Create a square patch with the color
    square = mpatches.Rectangle((i, 0), 1, 1, color=color)
    
    # Add the square to the plot
    ax.add_patch(square)
    
    # Add label in the center with white color
    ax.text(i + 0.5, 0.5, label, color='white', ha='center', va='center', fontsize=20)
    i += 1
# Set x-axis properties
ax.set_xlim(0, len(df_colors))
ax.set_xticks(range(len(df)))
ax.set_xticklabels([])  # Hide x-axis labels

# Set y-axis properties
ax.set_ylim(0, 1)
ax.set_yticks([])  # Hide y-axis labels
# Set plot title
plt.title('Color codes', fontsize=24)

plt.tight_layout()

plt.savefig(os.path.join(env_dict["DOWNLOADS_FOLDER"], "color_codes.pdf"), )

# Display the plot
plt.show()